In [ ]:
import warnings, time, os, json, itertools
warnings.filterwarnings("ignore")

from CombinedDataModule import *
from utils import *
from run_utils import *
from estimators import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from joblib import Parallel, delayed
from multiprocessing import cpu_count

In [ ]:
d = 2                           # 2-dim. covariate vector
n_MC = 10000                    # num. Monte Carlo samples to calculate ground truth mean in the target population
X_range = np.linspace(-1,1,51)  # range of the first covariate
U_range = np.linspace(-1,1,51)  # range of the first covariate

n_rct = 500     # trial sample size
n_tar = 20000   # target sample size
n_obs = 20000   # observational sample size

num_case_per_setting = 10               # num. different ground-truth cases to simulate for EACH GP param. setting
num_runs_per_case = 10                  # num. runs for EACH ground-truth case. a new trial sample is drawn every time.
num_est = 4                             # num. of estimators for the mean in the target population.
big_n_rct = num_runs_per_case * n_rct   # sample a big trial once and use its partitions in each run 

om_A0_par = {'ls':np.array([1,1]), 'alpha':[1,1]}  # unused rn.

om_A1_par_list = [{'ls':[0.25,0.5], 'alpha':[2,2]},
                  {'ls':[0.5,0.5], 'alpha':[2,2]}]

w_sel_par_list = [{'ls':[1,1e6], 'alpha':[1,0]},
                  {'ls':[1,1e6], 'alpha':[5,0]}]

w_trt_par_list = [{'ls':[0.25,0.5], 'alpha':[1,3]},
                  {'ls':[0.25,0.5], 'alpha':[1,10]}]

# om_A1_par_list = [{'ls':[0.25,0.5], 'alpha':[2,2]}]
# w_sel_par_list = [{'ls':[1,1e6], 'alpha':[1,0]}]
# w_trt_par_list = [{'ls':[0.25,0.5], 'alpha':[1,3]}]

np.random.seed(42)
case_seeds = np.random.randint(1e6, size=num_case_per_setting)

num_setting = len(om_A1_par_list) * len(w_sel_par_list) * len(w_trt_par_list)

avg_bias = np.zeros((num_setting, num_est))
std_bias = np.zeros((num_setting, num_est))
avg_var = np.zeros((num_setting, num_est))
std_var = np.zeros((num_setting, num_est))
avg_mse = np.zeros((num_setting, num_est))
std_mse = np.zeros((num_setting, num_est))

for ns, (om_A1_par, w_sel_par, w_trt_par) in enumerate(itertools.product(om_A1_par_list, w_sel_par_list, w_trt_par_list)):

    #  save the parameters for the setup.
    gp_setting_json = json.dumps({"om_A1_par": om_A1_par, "w_sel_par": w_sel_par, "w_trt_par": w_trt_par}, indent=4)
    save_dir = f"./results/gp_setting_{ns}"
    os.makedirs(save_dir, exist_ok=True)
    json_path = os.path.join(save_dir, f"gp_setting_{ns}.json")
    with open(json_path, 'w') as json_file:
        json_file.write(gp_setting_json)

    results= \
    Parallel(n_jobs=20)(delayed(sim_one_case) \
            (case_idx, random_seed, save_dir,  om_A0_par, om_A1_par, w_sel_par, w_trt_par, \
            d, big_n_rct, n_tar, n_obs, n_MC, num_est, num_runs_per_case, X_range, U_range) \
                    for case_idx, random_seed in enumerate(case_seeds))
    
    avg_bias[ns], std_bias[ns], avg_var[ns], std_var[ns], avg_mse[ns], std_mse[ns] = get_setting_stats(results)